In [1]:
import base64
import io
import openai
import os
import pandas as pd
import requests
import textwrap
from PIL import Image, ImageDraw, ImageFont

# OpenAI - API

In [2]:
# load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_timeout = 10

In [3]:
# list engines
engines = openai.Engine.list()

# print the first engine's id
print([i.id for i in engines.data])

['text-similarity-curie-001', 'gpt-4', 'gpt-4-0314', 'babbage', 'text-babbage-001', 'curie-similarity', 'davinci', 'code-davinci-edit-001', 'curie-instruct-beta', 'davinci-search-query', 'code-search-ada-code-001', 'text-curie-001', 'davinci-search-document', 'ada-code-search-text', 'text-davinci-001', 'text-search-davinci-query-001', 'code-search-ada-text-001', 'ada-code-search-code', 'babbage-002', 'babbage-search-document', 'text-search-curie-query-001', 'davinci-002', 'text-search-davinci-doc-001', 'text-embedding-ada-002', 'davinci-instruct-beta', 'davinci-similarity', 'ada-search-query', 'text-search-babbage-query-001', 'text-search-curie-doc-001', 'text-davinci-002', 'text-similarity-davinci-001', 'babbage-code-search-code', 'ada-search-document', 'text-search-babbage-doc-001', 'text-davinci-edit-001', 'gpt-3.5-turbo-0613', 'whisper-1', 'ada', 'code-search-babbage-code-001', 'text-ada-001', 'ada-similarity', 'code-search-babbage-text-001', 'gpt-4-0613', 'gpt-3.5-turbo-0301', 'cu

In [4]:
# TODO: list roles
role = "Your task is to write prompts that will be used in text to image models for generating memes. \
        Your response can have multiple prompts in the following format for each: a long prompt that describes a funny and agitative situtation starting with 'Prompt:', \
        then the caption of the corresponding meme that follows the prompt starting with 'Caption:'."
user = "Please try to come up with imaginative, very funny and novel memes, that describes specific situations. \
        Memes should be sarcastic and didactic, and should make laugh people while make them thinking about the society they live in. \
        Would be great to have the main themes about critism of capitalism, critism of commodity fetishism, alienation, critisim of while collar people and corporate culture, or class inequality. \
        Ideologically, these memes should be positioned in economic-left."

In [32]:
#df_prompt = pd.DataFrame(columns=["prompt", "caption", "seed"])
df_prompt = pd.read_csv("df_prompt.csv")
print(df_prompt.shape)

(20, 3)


In [6]:
def gpt_request():
    # ask the prompt
    response = ""
    try:
        response = openai.ChatCompletion.create(
            model= "gpt-3.5-turbo", # "gpt-4", "gpt-3.5-turbo",
            temperature=1,
            messages=[
                {"role": "system", "content": role},
                {"role": "user", "content": user},
            ]
        )
    except:
        pass
    # return result
    result = ''
    try:
        for choice in response.choices:
            result += choice.message.content
    except: 
        pass
    return result

In [7]:
#prompt = gpt_request()
#prompt

# Stability - API

In [8]:
# constants
url = "https://api.stability.ai/v1/generation/stable-diffusion-xl-1024-v1-0/text-to-image"

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": "sk-V0cNL6sSbOxye4pGeLYLUKwPv350WWmU2YGyHRajZVNnh0z4",
}

In [9]:
# TODO: set meme style
style = " The meme should be simple, featuring bold, caricatured characters with vivid colours, via very funny and imaginative imagery."

In [10]:
def stability_prompt_body(prompt):
    return {
        "steps": 40,
        "width": 1024,
        "height": 1024,
        "seed": 0,
        "cfg_scale": 5,
        "samples": 1,
        "text_prompts": [
            {
            "text": "Draw a meme which depicts: " + prompt + style,
            "weight": 1
            },
            {
                "text": "blurry, text, text box cloud",
                "weight": -1
            }
        ],
    }

# Image & Caption

In [11]:
def split_sentence(sentence):
    words = sentence.split()
    middle_index = len(words) // 2
    words.insert(middle_index, '\n')
    return ' '.join(words)

In [22]:
def add_caption_and_save(image, seed, caption):
    # create a new image with a white background, slightly taller to accommodate the caption
    width, height = image.size
    new_height = height + 100  # Adjust this value as needed
    new_image = Image.new("RGB", (width, new_height), "white")

    # paste the original image onto the new image
    new_image.paste(image, (0, 0))

    # create a drawing context on the new image
    draw = ImageDraw.Draw(new_image)

    # calculate text size and position based on font size and caption
    font = ImageFont.truetype("impact.ttf", 35)
    caption = caption.upper()
    caption_wrap = textwrap.wrap(caption, width=60)
    current_h, pad = height, 10
    for line in caption_wrap:
        w, h = draw.textsize(line, font=font)
        draw.text(((width - w) / 2, current_h), line, fill="black", font=font)
        current_h += h + pad

    # save the new image with the caption
    new_image.save(f'./memes/meme_{str(seed)}.png')

    # close the original image
    new_image.close()

In [23]:
"""
image = Image.open("memes/meme_2486965645.png")
seed = "2486965645"
caption = "Commodity Fetishism in the Modern Workplace: When productivity becomes a circus act and creativity is traded for chaos."
add_caption_and_save(image, seed, caption)
"""

# Generate Memes

In [33]:
# generate memes
for single_response in range(3):
    single_response = gpt_request()
    for i in single_response.split("Prompt:")[1:]:
        prompt = i.split("Caption:")[0].strip()
        caption = i.split("Caption:")[1].strip().replace('"', '')
        print("\nPrompt:", prompt)
        print("Caption:", caption)

        response = requests.post(
            url,
            headers=headers,
            json=stability_prompt_body(prompt),
        )

        if response.status_code != 200:
            raise Exception("Non-200 response: " + str(response.text))

        data = response.json()
        seed = data["artifacts"][0]["seed"]
        image = Image.open(io.BytesIO(base64.decodebytes(bytes(data["artifacts"][0]["base64"], "utf-8"))))
        add_caption_and_save(image, seed, caption)

        new_row = {'prompt': prompt, 'caption': caption, 'seed':seed}
        df_prompt = df_prompt.append(new_row, ignore_index=True)


Prompt: Imagine a world where money grows on trees, and instead of leaves, you see crisp dollar bills. The branches sway gently in the wind, tempting all who pass by. People are seen picking the money off the trees and stuffing it into their pockets. Your task is to create a meme that captures this situation, using an image and a hilarious caption.
Caption: When you finally find the perfect job: Tree Whisperer!


C:\Users\san-p\AppData\Local\Temp\ipykernel_52244\300866474.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prompt = df_prompt.append(new_row, ignore_index=True)



Prompt: Picture a bustling office where employees are sitting at their desks, typing away furiously. Suddenly, a giant inflatable rat bursts through the office walls and starts rampaging around, causing chaos and knocking over cubicles. Interact with the scenario and create a clever meme that shows the absurdity of corporate culture.
Caption: When your office pet gets out of control: Meet Mr. Inflation, the Disgruntled Rat!


C:\Users\san-p\AppData\Local\Temp\ipykernel_52244\300866474.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prompt = df_prompt.append(new_row, ignore_index=True)



Prompt: Visualize a world where people wear suits made entirely out of branded logos, from head to toe. The streets are littered with name tag stickers, with each person proudly displaying their favorite corporation. In this logo-obsessed society, create a meme that highlights the extreme commodification of our daily lives.
Caption: When being fashionably corporate means wearing everything but your true identity!


...

C:\Users\san-p\AppData\Local\Temp\ipykernel_52244\300866474.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prompt = df_prompt.append(new_row, ignore_index=True)



Prompt: In a world where individuality is suppressed, a meme reflects the struggle to maintain authenticity in a conformist society.
Caption: Me trying to fit into society's mold... but I'm a stubborn piece of Play-Doh.


C:\Users\san-p\AppData\Local\Temp\ipykernel_52244\300866474.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prompt = df_prompt.append(new_row, ignore_index=True)



Prompt: A meme mocks the concept of work-life balance, highlighting the demanding nature of modern jobs.
Caption: When 'working from home' means answering emails from your boss while rewatching every season of 'Friends'.


C:\Users\san-p\AppData\Local\Temp\ipykernel_52244\300866474.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prompt = df_prompt.append(new_row, ignore_index=True)


In [34]:
df_prompt

,prompt,caption,seed
0,"In a world ruled by capitalism, a meme that po...",Capitalism: Making sure the rich never miss a ...,530730371
1,Picture a chaotic office environment with empl...,Commodity Fetishism in the Modern Workplace: W...,4079648354
2,Imagine a meme with a line of identical worker...,Breaking Free from the Assembly Line of Confor...,3721060941
3,Picture a meme showing a group of corporate ex...,Corporate Culture: Where the red nose is not j...,55447854
4,Envision a meme with a large-scale mansion loo...,Class Inequality: The precariousness of the wo...,943380404
5,Imagine a meme with a conveyor belt in a facto...,White Collar Life: When a moving assembly line...,1561050219
6,Picture a meme showing a stack of money with a...,The Loneliness of Wealth: When the pursuit of ...,3001369557
7,"In a world where suits are the norm, imagine a...",Comfy Revolution: Rejecting the notion that pr...,878875540
8,"Picture a meme showing a broken escalator, wit...",The Illusion of Upward Mobility: When progress...,1464511519
9,Envision a meme depicting a swarm of office wo...,Chasing Corporate Success: When the elusive re...,4193377371


In [35]:
# save all prompts
df_prompt.to_csv("df_prompt.csv", index=False)